In [ ]:
import os
os.chdir("C:\\Users\\erin.melly\\Desktop\\GitHub\\Tax-Calculator")
import behresp
import microdf as mdf
from taxcalc import *
import pandas as pd
import numpy as np
import copy

In [ ]:
# read in PUF
puf_df = pd.read_csv(r'C:\Users\erin.melly\Desktop\GitHub\Tax-Calculator\PUF.csv')

# set year
cyr = 2027

# elasticity dictionary 
cg= {2021: -4.184,
     2022: -3.488,
     2023: -2.76,
     2024: -2.76,
     2025: -2.76,
     2026: -2.76,
     2027: -2.76,
     2028: -2.76,
     2029: -2.76,
     2030: -2.76
    }
# capital gains realization at death dictionary
realization_at_death = {2021: 204253570343, 
                        2022: 215627184994,
                        2023: 226688939254, 
                        2024: 235220544654,
                        2025: 243736672365,
                        2026: 245533163473,
                        2027: 254865207631,
                        2028: 264380671657,
                        2029: 274178230759,
                        2030: 284077457991
                       }

In [ ]:
# create new variable for gains taxed at death in PUF

# share of capital gains
puf_df = puf_df.assign(ltgains_weight = (puf_df['p23250'] * puf_df['s006']) / sum(puf_df['p23250'] * puf_df['s006']))
#puf_df.loc[(puf_df['p23250'] > 0), 'ltgains_weight'] = (puf_df.loc[(puf_df['p23250'] > 0), 'p23250'] * puf_df.loc[(puf_df['p23250'] > 0), 's006']) \
#/ sum(puf_df.loc[(puf_df['p23250'] > 0), 'p23250'] * puf_df.loc[(puf_df['p23250'] > 0), 's006'])
# assign total realization at death revenue from JCT to taxpayers based on capital gains weight
puf_df['gains_at_death'] = (puf_df['ltgains_weight'] * realization_at_death[cyr]) / (puf_df['s006'] / 100) 

In [ ]:
# current law calculator
rec = Records(data= puf_df)
pol = Policy()
calc= Calculator(pol, rec)
calc.advance_to_year(cyr)
calc.calc_all()
df = calc.dataframe([], all_vars=True)
rev1static = calc.weighted_total('combined')

In [ ]:
# Revenue Change from Reform, with Behavioral Responses

# specify Calculator object for static analysis of reform policy
pol.implement_reform(Policy.read_json_reform('taxcalc\\reforms\\Biden.json'))
calc2 = Calculator(pol, rec)

# calculate reform income tax liabilities for cyr under static assumptions
calc2.advance_to_year(cyr)
calc2.calc_all()
rev2static = calc2.weighted_total('combined')

# specify assumed non-zero response-function substitution elasticity
response_elasticities = {'sub': 0.25,
                         'cg': cg[cyr]}

# specify Calculator object for analysis of reform with behavioral responses
calc2 = Calculator(policy=pol, records=rec)
calc2.advance_to_year(cyr)
_, df2br = behresp.response(calc, calc2, response_elasticities)

# calculate reform income tax liabilities for cyr with behavioral response
rev2br = (df2br['combined'] * df2br['s006']).sum()

# print total income tax revenue estimates for cyr (estimates in billons of dollars)
print('{}_DYNAMIC_DIFERENCE_tot_rev($B)= {:.3f}'.format(cyr, (rev2br - rev1static) * 1e-9))